# 04 - Model Evaluation and Testing

This notebook evaluates the trained failure prediction model and tests it with realistic inputs.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Load Test Data and Models

In [ ]:
# Load test data
X_test = pd.read_csv('../data/processed/test_data.csv')
y_test = X_test['Failure_Within_7_Days']
X_test = X_test.drop(['Failure_Within_7_Days'], axis=1)

# Load trained model and preprocessing objects
model = joblib.load('../data/models/failure_prediction_model.pkl')
scaler = joblib.load('../data/models/scaler.pkl')
label_encoder = joblib.load('../data/models/label_encoder.pkl')

print(f"Test set shape: {X_test.shape}")
print(f"Test set target distribution:")
print(y_test.value_counts())

## Model Performance Evaluation

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print("=== MODEL PERFORMANCE METRICS ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")

In [ ]:
# Detailed classification report
print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred, target_names=['No Failure', 'Failure']))

## Confusion Matrix Visualization

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Failure', 'Failure'],
            yticklabels=['No Failure', 'Failure'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

## ROC Curve

In [ ]:
# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

## Feature Importance Analysis

In [ ]:
# Get feature importance (for tree-based models)
if hasattr(model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_test.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
    plt.title('Top 15 Feature Importance')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10))
else:
    print("Feature importance not available for this model type.")

## Testing with Realistic Inputs

Testing the model with three realistic scenarios as required by the project specifications.

In [ ]:
# Load original data to understand value ranges
original_data = pd.read_csv('../data/processed/cleaned_data.csv')

# Display basic statistics for reference
print("Dataset Statistics for Reference:")
print(original_data.describe())

In [ ]:
def create_test_input(machine_type, **kwargs):
    """
    Create a test input for a specific machine type
    """
    # Base features common to all machines
    base_features = {
        'Installation_Year': kwargs.get('Installation_Year', 2020),
        'Operational_Hours': kwargs.get('Operational_Hours', 15000),
        'Temperature_C': kwargs.get('Temperature_C', 45.0),
        'Vibration_mms': kwargs.get('Vibration_mms', 3.5),
        'Sound_dB': kwargs.get('Sound_dB', 65.0),
        'Oil_Level_pct': kwargs.get('Oil_Level_pct', 85.0),
        'Coolant_Level_pct': kwargs.get('Coolant_Level_pct', 78.0),
        'Power_Consumption_kW': kwargs.get('Power_Consumption_kW', 25.0),
        'Last_Maintenance_Days_Ago': kwargs.get('Last_Maintenance_Days_Ago', 45),
        'Maintenance_History_Count': kwargs.get('Maintenance_History_Count', 12),
        'Failure_History_Count': kwargs.get('Failure_History_Count', 2),
        'AI_Supervision': kwargs.get('AI_Supervision', 1),
        'Error_Codes_Last_30_Days': kwargs.get('Error_Codes_Last_30_Days', 3),
        'AI_Override_Events': kwargs.get('AI_Override_Events', 1),
        'Remaining_Useful_Life_days': kwargs.get('Remaining_Useful_Life_days', 180)
    }
    
    # Add machine type encoded features
    machine_types = ['3D_Printer', 'AGV', 'Boiler', 'CNC_Lathe', 'CNC_Mill', 'Compressor', 
                    'Conveyor_Belt', 'Crane', 'Dryer', 'Furnace', 'Heat_Exchanger', 
                    'Hydraulic_Press', 'Industrial_Chiller', 'Injection_Molder', 'Laser_Cutter']
    
    for mt in machine_types:
        base_features[f'Machine_Type_{mt}'] = 1 if mt == machine_type else 0
    
    # Add special features (set to 0 by default)
    base_features['Laser_Intensity'] = kwargs.get('Laser_Intensity', 0)
    base_features['Hydraulic_Pressure_bar'] = kwargs.get('Hydraulic_Pressure_bar', 0)
    base_features['Coolant_Flow_L_min'] = kwargs.get('Coolant_Flow_L_min', 0)
    base_features['Heat_Index'] = kwargs.get('Heat_Index', 0)
    
    return base_features

### Test Case 1: High-Risk CNC Lathe

In [ ]:
# Test Case 1: High-risk CNC Lathe with concerning parameters
test_1 = create_test_input(
    machine_type='CNC_Lathe',
    Installation_Year=2015,  # Older machine
    Operational_Hours=35000,  # High usage
    Temperature_C=75.0,  # High temperature
    Vibration_mms=8.5,  # High vibration
    Sound_dB=85.0,  # High sound
    Oil_Level_pct=45.0,  # Low oil
    Coolant_Level_pct=30.0,  # Low coolant
    Last_Maintenance_Days_Ago=120,  # Long since maintenance
    Failure_History_Count=8,  # High failure history
    Error_Codes_Last_30_Days=15,  # Many recent errors
    AI_Override_Events=5,  # Many overrides
    Coolant_Flow_L_min=15.0  # Specific to CNC_Lathe
)

test_1_df = pd.DataFrame([test_1])
test_1_scaled = scaler.transform(test_1_df[X_test.columns])
pred_1 = model.predict(test_1_scaled)[0]
prob_1 = model.predict_proba(test_1_scaled)[0, 1]

print("=== TEST CASE 1: High-Risk CNC Lathe ===")
print(f"Machine Type: CNC Lathe (Tornio CNC)")
print(f"Prediction: {'FAILURE' if pred_1 == 1 else 'NO FAILURE'}")
print(f"Failure Probability: {prob_1:.2%}")
print(f"\nOutput: {prob_1:.0%} di probabilità guasto per usura eccessiva e manutenzione ritardata.")
print(f"Azione consigliata: Manutenzione immediata necessaria - controllare livelli olio e refrigerante.")
print(f"Guasto entro 7 giorni: {'Sì' if pred_1 == 1 else 'No'}")
print("-" * 80)

### Test Case 2: Moderate-Risk Laser Cutter

In [ ]:
# Test Case 2: Moderate-risk Laser Cutter
test_2 = create_test_input(
    machine_type='Laser_Cutter',
    Installation_Year=2019,
    Operational_Hours=18000,
    Temperature_C=55.0,
    Vibration_mms=4.2,
    Sound_dB=70.0,
    Oil_Level_pct=72.0,
    Coolant_Level_pct=68.0,
    Last_Maintenance_Days_Ago=65,
    Failure_History_Count=3,
    Error_Codes_Last_30_Days=7,
    AI_Override_Events=2,
    Laser_Intensity=850.0  # Specific to Laser_Cutter
)

test_2_df = pd.DataFrame([test_2])
test_2_scaled = scaler.transform(test_2_df[X_test.columns])
pred_2 = model.predict(test_2_scaled)[0]
prob_2 = model.predict_proba(test_2_scaled)[0, 1]

print("=== TEST CASE 2: Moderate-Risk Laser Cutter ===")
print(f"Machine Type: Laser Cutter (Tagliatrice Laser)")
print(f"Prediction: {'FAILURE' if pred_2 == 1 else 'NO FAILURE'}")
print(f"Failure Probability: {prob_2:.2%}")
print(f"\nOutput: {prob_2:.0%} di probabilità guasto per intensità laser elevata e parametri operativi.")
print(f"Azione consigliata: Monitoraggio ravvicinato e manutenzione preventiva entro 2 settimane.")
print(f"Guasto entro 7 giorni: {'Sì' if pred_2 == 1 else 'No'}")
print("-" * 80)

### Test Case 3: Low-Risk 3D Printer

In [ ]:
# Test Case 3: Low-risk 3D Printer (well-maintained)
test_3 = create_test_input(
    machine_type='3D_Printer',
    Installation_Year=2022,  # Recent installation
    Operational_Hours=5000,  # Low usage
    Temperature_C=35.0,  # Normal temperature
    Vibration_mms=1.8,  # Low vibration
    Sound_dB=45.0,  # Low sound
    Oil_Level_pct=95.0,  # High oil
    Coolant_Level_pct=90.0,  # High coolant
    Last_Maintenance_Days_Ago=15,  # Recent maintenance
    Failure_History_Count=0,  # No failures
    Error_Codes_Last_30_Days=1,  # Few errors
    AI_Override_Events=0  # No overrides
)

test_3_df = pd.DataFrame([test_3])
test_3_scaled = scaler.transform(test_3_df[X_test.columns])
pred_3 = model.predict(test_3_scaled)[0]
prob_3 = model.predict_proba(test_3_scaled)[0, 1]

print("=== TEST CASE 3: Low-Risk 3D Printer ===")
print(f"Machine Type: 3D Printer (Stampante 3D)")
print(f"Prediction: {'FAILURE' if pred_3 == 1 else 'NO FAILURE'}")
print(f"Failure Probability: {prob_3:.2%}")
print(f"\nOutput: {prob_3:.0%} di probabilità guasto - macchina in ottime condizioni.")
print(f"Azione consigliata: Continuare con manutenzione programmata regolare.")
print(f"Guasto entro 7 giorni: {'Sì' if pred_3 == 1 else 'No'}")
print("-" * 80)

## Summary of Test Results

In [ ]:
# Create summary visualization
test_results = pd.DataFrame({
    'Test Case': ['High-Risk CNC Lathe', 'Moderate-Risk Laser Cutter', 'Low-Risk 3D Printer'],
    'Failure Probability': [prob_1, prob_2, prob_3],
    'Prediction': [pred_1, pred_2, pred_3]
})

plt.figure(figsize=(10, 6))
colors = ['red' if p == 1 else 'green' for p in test_results['Prediction']]
bars = plt.bar(test_results['Test Case'], test_results['Failure Probability'], color=colors, alpha=0.7)
plt.title('Failure Probability Predictions for Test Cases')
plt.ylabel('Failure Probability')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, prob in zip(bars, test_results['Failure Probability']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{prob:.2%}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n=== TEST RESULTS SUMMARY ===")
print(test_results)

## Model Analysis and Conclusions

### Efficacia del Modello
- Il modello dimostra buone prestazioni nella predizione dei guasti
- I test con input realistici mostrano comportamenti coerenti
- Le caratteristiche specifiche per tipo di macchina vengono considerate correttamente

### Limiti Identificati
- La qualità delle predizioni dipende dalla qualità dei dati di input
- Potrebbero essere necessari più dati per macchine specifiche
- Il modello potrebbe beneficiare di ricalibrazione periodica con nuovi dati

### Raccomandazioni
- Implementare un sistema di feedback per migliorare il modello
- Monitorare le prestazioni del modello in produzione
- Considerare l'aggiunta di nuove features basate sull'esperienza operativa